# Quick Map

In [1]:
from importlib import reload
import pathlib as pl
import geopandas as gpd
from IPython.display import IFrame  

In [2]:
import snippets
reload(snippets)
from snippets import *

### Read in data, projections, etc.

In [3]:
rootdir = pl.Path(r'Y:\CCSI\TECH\FEMA\2019_PFRA\DATA\NJ\Spatialfiles\Scope\html_map')
crs = '+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs' 

In [4]:
gages = gpd.read_file(str(rootdir/'gages.geojson'))
gages = quick_transform(gages, crs)

rivers = gpd.read_file(str(rootdir/'rivers.geojson'))
rivers = quick_transform(rivers, crs)

huc12s = gpd.read_file(str(rootdir/'huc12s.geojson'))
huc12s = quick_transform(huc12s, crs)

### Assign Map Center

In [5]:
xcoord, ycoord = get_centroid(gages)
m = interactive_map(ycoord, xcoord)

### For each shape added to the map, assign columnar data to show in pop-up

In [6]:
gages.columns

Index(['OBJECTID', 'Gage_ID', 'Agency_CD', 'Site_NO', 'Station_NM',
       'Site_TP_CD', 'ReachCode', 'Stream_ID', 'State', 'State_FIPS', 'Region',
       'HUC8', 'Lat_NWIS', 'Long_NWIS', 'Lat_1M', 'Long_1M', 'Source',
       'Review', 'Measure', 'NWISWeb', 'geometry'],
      dtype='object')

In [7]:
gages_info = {'Gage ID':'Gage_ID', 'Station':'Station_NM', 'URL':'NWISWeb'}
m = add_point_popups(gages, m, gages_info)

### Make popup for lines

In [8]:
rivers.columns

Index(['OBJECTID', 'REACH_ID', 'UP_NODE', 'DN_NODE', 'ROUTE_METH',
       'SOURCE_CIT', 'SHAPE_Length', 'geometry'],
      dtype='object')

In [9]:
rivers_info = {'REACH ID':'REACH_ID'}
m = add_line_popups(rivers, m, rivers_info)

### Make popup for polygons

In [10]:
huc12s.columns

Index(['OBJECTID', 'HUC12', 'NAME_1', 'PTS', 'Company', 'Sqmi', 'Domains',
       'Group_D', 'OBJECTID_1__', 'Label', 'Label_1', 'Pluvial', 'Fluvial',
       'PM', 'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object')

In [11]:
def add_poly_popups(gdf0, folmap, info):
    sID='Hi'
    for idx in  gdf0.index:
        html_string = f"<h4>{sID}</h4>"
        
        for k, v in info.items():
            if k == 'URL':
                add_data = f'<a href={gdf0.loc[idx][v]} target="_blank" >USGS Webpage   </a>'
                html_string +=add_data
            else:
                add_data = f"<p>{k} {gdf0.loc[idx][v]}</p>"
                html_string +=add_data
        iframe=branca.element.IFrame(html=html_string, width=200, height=250)    
        popup = folium.Popup(iframe)
        #for poly in gdf0.geometry[idx]:
        #    x = poly.exterior.coords.xy[0]
        #    y = poly.exterior.coords.xy[1]
        #    line_points = zip(y,x)
        #    print(list(line_points))
        style_function = lambda x :{'fillColor': '#a6a6a6','color': '#a6a6a6','opacity':0.1,'fillOpacity': 0.01}
        geojson = folium.GeoJson(gdf0,style_function=style_function)
        geojson.add_child(popup)
        geojson.add_to(folmap)
    return folmap

In [12]:
huc12s_info = {'HUC12':'HUC12', 'Domain':'Domains', 'Label':'Label', 'PM':'PM'}
m = add_poly_popups(huc12s, m, huc12s_info)

In [13]:
m = add_tiles(m)

### save to html

In [14]:
m.save(outfile='map.html')  
IFrame('map.html', width=800, height=650)

# END